# Tweet Improvement Workflow Summary

| Step           | Description                                                  |
|----------------|--------------------------------------------------------------|
| Generate Tweet | AI creates a funny tweet on a given topic                    |
| Evaluate Tweet | AI scores tweet 0-10 and provides feedback                   |
| Route Decision | If score ≥ 7 or max iterations reached → approve; else optimize |
| Optimize Tweet | AI rewrites tweet to improve humor and virality              |
| Repeat Loop    | Runs evaluate → optimize until approved or max iterations    |
| Final Output   | Returns approved tweet and history of all tweet versions     |

---

**Note:**  
If initial tweet is good, workflow stops early with no further optimization. To see improvements over iterations, lower score threshold or increase max iterations.


In [15]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
# Initialize Gemini LLMs
generator_llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash")
evaluator_llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash")
optimizer_llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash")


In [17]:
class TweetEvaluation(BaseModel):
    score: int = Field(..., ge=0, le=10, description="Score out of 10")
    feedback: str = Field(..., description="Detailed feedback")

structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)


In [18]:
class TweetState(TypedDict):
    topic: str
    tweet: str
    score: int
    feedback: str
    iteration: int
    max_iteration: int
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
    response: str 

In [19]:
def generate_tweet(state: TweetState):
    prompt = f"""You are a funny and clever Twitter influencer.

Write a short, original, hilarious tweet on the topic: "{state['topic']}".
- Max 280 characters.
- Use observational humor, irony, sarcasm, meme logic.
- Avoid question-answer format.

Output just the tweet."""
    response = generator_llm.invoke(prompt).content
    return {'tweet': response, 'tweet_history': [response]}

def evaluate_tweet(state: TweetState):
    prompt = f"""Rate this tweet on scale 0-10 for humor, originality, and virality:

Tweet: "{state['tweet']}"

Respond ONLY with JSON with keys: score (int), feedback (string)."""
    output = structured_evaluator_llm.invoke(prompt)
    return {'score': output.score, 'feedback': output.feedback, 'feedback_history': [output.feedback]}

def optimize_tweet(state: TweetState):
    prompt = f"""Improve this tweet to be funnier and more viral based on this feedback:

"{state['feedback']}"

Original tweet:
{state['tweet']}

Output only the improved tweet, max 280 characters."""
    response = optimizer_llm.invoke(prompt).content
    return {'tweet': response, 'iteration': state['iteration'] + 1, 'tweet_history': [response]}

def route_evaluation(state: TweetState):
    if state['score'] >= 7 or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

def approved(state: TweetState):
    return {'response': f"Final approved tweet after {state['iteration']} iterations:\n{state['tweet']}"}

In [20]:
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)
graph.add_node('approved', approved)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved': 'approved', 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')
graph.add_edge('approved', END)

workflow = graph.compile()

In [23]:
initial_state = {
    'topic': 'Artificial Intelligence in Daily Life',
    'tweet': '',
    'score': 0,
    'feedback': '',
    'iteration': 2,
    'max_iteration': 5,
    'tweet_history': [],
    'feedback_history': [],
    'response': ''  # Initialize response key to avoid missing keys
}

result = workflow.invoke(initial_state)

print(result['response'])

Final approved tweet after 2 iterations:
We were promised AI that could cure diseases and end war. Instead, my smart fridge just asked if I wanted to "optimize my snack experience." Priorities, people. Priorities.


In [22]:
#  print all versions of the tweet
print("\nTweet History:")
for t in result['tweet_history']:
    print(t)


Tweet History:
AI can now generate hyper-realistic images and compose symphonies, yet my smart fridge still thinks I'm out of milk when I'm staring directly at it. It's giving "genius in theory, chaos in practice." #AIDailyLife
